In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('chained_assignment',None)

In [2]:
fpath_listings_detailed = '../SanDiego/listings_detailed.csv'

list_df = pd.read_csv(fpath_listings_detailed)
print list_df.shape

(6608, 95)


In [3]:
features = ['price','property_type','room_type','accommodates','bathrooms',
            'bedrooms','beds','bed_type','neighbourhood_cleansed','latitude','longitude',
            'amenities']

# to consider:
# 'square_feet' -- lots of NaNs

pre_df = list_df[features]
print pre_df.shape
pre_df.head()

(6608, 12)


price property_type        room_type  accommodates  bathrooms  bedrooms  \
0   $49.00         House     Private room             2        1.0       1.0   
1   $59.00   Condominium     Private room             2        1.0       1.0   
2  $150.00   Condominium  Entire home/apt             6        2.5       3.0   
3  $120.00     Townhouse  Entire home/apt             4        1.5       2.0   
4   $70.00         House     Private room             2        1.0       1.0   

   beds  bed_type       neighbourhood_cleansed   latitude   longitude  \
0   1.0  Real Bed  Amphitheater And Water Park  32.581882 -117.001940   
1   1.0  Real Bed                    East Lake  32.653029 -116.976812   
2   3.0  Real Bed                    East Lake  32.661477 -116.971813   
3   3.0  Real Bed              Eastlake Trails  32.638694 -116.951252   
4   1.0  Real Bed              Eastlake Vistas  32.638942 -116.938891   

                                           amenities  
0  {TV,"Wireless Internet",Heating,"Family/Kid Fr...  
1  {Internet,"Wireless Internet","Air Conditionin...  
2  {TV,"Cable TV","Wireless Internet","Air Condit...  
3  {TV,"Cable TV",Internet,"Wireless Internet",Ki...  
4  {TV,"Cable TV",Internet,"Wireless Internet",Po...

In [4]:
def amenity_list(x):
    amenities_str = x.replace('"','').replace('{','').replace('}','')
    amenities_list = amenities_str.split(',')
    return amenities_list

def price_to_float(x):
    x = x.replace('$','')
    x = x.replace(',','')
    return float(x)

pre_df['amenities'] = pre_df['amenities'].apply(amenity_list)
pre_df['price'] = pre_df['price'].apply(price_to_float)

print pre_df.shape
pre_df.head()

(6608, 12)


price property_type        room_type  accommodates  bathrooms  bedrooms  \
0   49.0         House     Private room             2        1.0       1.0   
1   59.0   Condominium     Private room             2        1.0       1.0   
2  150.0   Condominium  Entire home/apt             6        2.5       3.0   
3  120.0     Townhouse  Entire home/apt             4        1.5       2.0   
4   70.0         House     Private room             2        1.0       1.0   

   beds  bed_type       neighbourhood_cleansed   latitude   longitude  \
0   1.0  Real Bed  Amphitheater And Water Park  32.581882 -117.001940   
1   1.0  Real Bed                    East Lake  32.653029 -116.976812   
2   3.0  Real Bed                    East Lake  32.661477 -116.971813   
3   3.0  Real Bed              Eastlake Trails  32.638694 -116.951252   
4   1.0  Real Bed              Eastlake Vistas  32.638942 -116.938891   

                                           amenities  
0  [TV, Wireless Internet, Heating, Family/Kid Fr...  
1  [Internet, Wireless Internet, Air Conditioning...  
2  [TV, Cable TV, Wireless Internet, Air Conditio...  
3  [TV, Cable TV, Internet, Wireless Internet, Ki...  
4  [TV, Cable TV, Internet, Wireless Internet, Po...

In [5]:
# get binary for amenities
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
amenity_mlb = mlb.fit_transform(pre_df.amenities)
amenity_df = pd.DataFrame(amenity_mlb, columns=mlb.classes_)

# remove the weird columns
amenity_df.drop('', axis=1, inplace=True)
amenity_df.drop('translation missing: en.hosting_amenity_49', axis=1, inplace=True)
amenity_df.drop('translation missing: en.hosting_amenity_50', axis=1, inplace=True)
print amenity_df.shape
amenity_df.head()

# concatenate dataframe
concat_df = pd.concat([pre_df,amenity_df],axis=1)
concat_df.drop('amenities', axis=1, inplace=True)
concat_df.head()

# get dummies
sql_df = pd.get_dummies(concat_df)

# print df shape
print sql_df.shape
sql_df.dropna(axis=0, how='any', inplace=True) # drop nan rows
print sql_df.shape

(6608, 40)
(6608, 172)
(6593, 172)


In [6]:
sql_df.head()

price  accommodates  bathrooms  bedrooms  beds   latitude   longitude  \
0   49.0             2        1.0       1.0   1.0  32.581882 -117.001940   
1   59.0             2        1.0       1.0   1.0  32.653029 -116.976812   
2  150.0             6        2.5       3.0   3.0  32.661477 -116.971813   
3  120.0             4        1.5       2.0   3.0  32.638694 -116.951252   
4   70.0             2        1.0       1.0   1.0  32.638942 -116.938891   

   24-Hour Check-in  Air Conditioning  Breakfast  \
0                 0                 0          0   
1                 0                 1          0   
2                 0                 1          0   
3                 1                 0          0   
4                 1                 0          1   

                  ...                  neighbourhood_cleansed_Thomy Locust Pl  \
0                 ...                                                       0   
1                 ...                                                       0   
2                 ...                                                       0   
3                 ...                                                       0   
4                 ...                                                       0   

   neighbourhood_cleansed_Tierrasanta  \
0                                   0   
1                                   0   
2                                   0   
3                                   0   
4                                   0   

   neighbourhood_cleansed_Tijuana River Valley  \
0                                            0   
1                                            0   
2                                            0   
3                                            0   
4                                            0   

   neighbourhood_cleansed_Torrey Pines  \
0                                    0   
1                                    0   
2                                    0   
3                                    0   
4                                    0   

   neighbourhood_cleansed_University City  \
0                                       0   
1                                       0   
2                                       0   
3                                       0   
4                                       0   

   neighbourhood_cleansed_Valencia Park  neighbourhood_cleansed_Webster  \
0                                     0                               0   
1                                     0                               0   
2                                     0                               0   
3                                     0                               0   
4                                     0                               0   

   neighbourhood_cleansed_West University Heights  \
0                                               0   
1                                               0   
2                                               0   
3                                               0   
4                                               0   

   neighbourhood_cleansed_Wooded Area  neighbourhood_cleansed_Yosemite Dr  
0                                   0                                   0  
1                                   0                                   0  
2                                   0                                   0  
3                                   0                                   0  
4                                   0                                   0  

[5 rows x 172 columns]

# SQL Section

In [9]:
from sqlalchemy import create_engine

disk_engine = create_engine('sqlite:////Users/Greg/Projects/AirBnB/airbnb.db')
sql_df.to_sql('properties', disk_engine, if_exists='replace', index=False)